"""
Created on Sun May 28 19:51:21 2023

@author: gakro
"""

this program is to predict the final position of each driver in the race.
thus, the output should be the position (ordinal data from 1 to 20).
the inputs (which parameters to use and which year, race, and session to use)
and the appropriate algorithms should be considered.

training set: Bahrain, Saudi Arabia, Australia, Azerbaijan (all in 2023)
test set: Miami (2023)

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from fastf1 import core
import fastf1

the following block of code loads the session data of each qualifying race

In [16]:
bahrain_quali = fastf1.get_session(2023, 1, 4)
bahrain_quali.load(telemetry=True, laps=True, weather=True)

saudi_quali = fastf1.get_session(2023, 2, 4)
saudi_quali.load(telemetry=True, laps=True, weather=True)

australia_quali = fastf1.get_session(2023, 3, 4)
australia_quali.load(telemetry=True, laps=True, weather=True)

azerbaijan_quali = fastf1.get_session(2023, 4, 4)
azerbaijan_quali.load(telemetry=True, laps=True, weather=True)

miami_quali = fastf1.get_session(2023, 5, 4)
miami_quali.load(telemetry=True, laps=True, weather=True)

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	Using cached data for season_schedule
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.0.3]
req            INFO 	Using cached data for driver_info
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	D

the following block of code loads the session data of each race

In [17]:
bahrain_race = fastf1.get_session(2023, 1, 5)
bahrain_race.load(telemetry=True, laps=True, weather=True)

saudi_race = fastf1.get_session(2023, 2, 5)
saudi_race.load(telemetry=True, laps=True, weather=True)

australia_race = fastf1.get_session(2023, 3, 5)
australia_race.load(telemetry=True, laps=True, weather=True)

azerbaijan_race = fastf1.get_session(2023, 4, 5)
azerbaijan_race.load(telemetry=True, laps=True, weather=True)

miami_race = fastf1.get_session(2023, 5, 5)
miami_race.load(telemetry=True, laps=True, weather=True)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.0.3]
req            INFO 	Using cached data for driver_info
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetchin

the following code extracts the qualifying and race results for each race. grand prix name_session name_results stores each driver's car numbers in the order of their finishing result.

In [25]:
bahrain_quali_result = bahrain_quali.results.iloc[:,0]
saudi_quali_result = saudi_quali.results.iloc[:,0]
australia_quali_result = australia_quali.results.iloc[:,0]
azerbaijan_quali_result = azerbaijan_quali.results.iloc[:,0]
miami_quali_result = miami_quali.results.iloc[:,0]

bahrain_race_result = bahrain_race.results.iloc[:,0]
saudi_race_result = saudi_race.results.iloc[:,0]
australia_quali_result = australia_race.results.iloc[:,0]
azerbaijan_quali_result = azerbaijan_race.results.iloc[:,0]
miami_quali_result = miami_race.results.iloc[:,0]

plot the quali position in the x axis and the race finishing position in the y axis
to see how much they are correlated as the starter.
Try to use the logistic regressio. However, it must be for multiple class
(since what we are predicting is the position in the end of the race from 1 to 20),
and for ordinal output (since the position has the order).

By default, the session results are indexed by driver number and sorted by finishing position.

<class 'pandas.core.series.Series'>
